# Dolování ISBN a ISSN z Alephu

ISBN a ISSN jsou v ALephu součástí BIB záznamu - https://knowledge.exlibrisgroup.com/Aleph/Knowledge_Articles/How_to_export_the_BIB_record_from_Aleph


Jednotlivé záznamy v exportu se ve smyčce procházejí - budeme hledat:
- controlfield tag="001 - sysno
- datafield 020 - podpole a - ISBN
- datafield 022 - podpole a - ISSN 
- datafield 245 - podpole a - hlavní název

In [1]:
import re
import pandas as pd

In [2]:
with open('msweep-export-all_plný.txt', 'r', encoding="utf8", errors="ignore") as f:
    data = f.read()


In [5]:
#zobrazení prvního záznamu, re.DOTALL umožní srovnání vzoru s víceřádkovým rozpětím
zaznamy = re.findall("<record.+?>(.+?)</record>", data, re.DOTALL)
print(zaznamy[10])

00000cam-a22-----7a-4500</leader>
    <controlfield tag='001'>000000011</controlfield>
    <controlfield tag='003'>CZ-BrMU</controlfield>
    <controlfield tag='005'>20151029083923.0</controlfield>
    <controlfield tag='008'>021119s2002----xr-|||||-|||||||||||cze-d</controlfield>
    <datafield tag='015' ind1='' ind2=''>
      <subfield code='a'>cnb001181661</subfield>
    </datafield>
    <datafield tag='020' ind1='' ind2=''>
      <subfield code='a'>80-86515-15-X</subfield>
    </datafield>
    <datafield tag='040' ind1='' ind2=''>
      <subfield code='a'>BOD001</subfield>
      <subfield code='b'>cze</subfield>
      <subfield code='d'>BOD114</subfield>
    </datafield>
    <datafield tag='072' ind1='' ind2='7'>
      <subfield code='a'>94(437)</subfield>
      <subfield code='x'>Dějiny Česka a Slovenska</subfield>
      <subfield code='2'>Konspekt</subfield>
      <subfield code='9'>8</subfield>
    </datafield>
    <datafield tag='100' ind1='1' ind2=''>
      <subfield code='a'>

In [4]:
#kontrola sysna
sysno = re.findall("<controlfield tag='001'>(\d+)</controlfield>", zaznamy[10])
print(sysno)

['000000011']


In [9]:
#kontrola ISBN
ISBN = re.findall("<datafield tag='020'.+?<subfield code='a'>(.+?)</subfield>", zaznamy[10], re.DOTALL )
print(ISBN)

['80-86515-15-X']


In [ ]:
#kontrola ISSN
ISSN = re.findall("datafield tag='022'.+?<subfield code='a'>(.+?)</subfield>", zaznamy[10], re.DOTALL)
print(ISSN)

In [ ]:
#kontrola názvu
nazev = re.findall("<datafield tag='245'.+?<subfield code='a'>(.+?)</subfield>", zaznamy[10], re.DOTALL)
print(nazev[0])

In [ ]:
sloupce=["sysno", "ISBN", "ISSN", "nazev"]
zaznamy_aleph = pd.DataFrame(columns = sloupce)
zaznamy_aleph.head()

In [ ]:
i = 0
for zaznam in zaznamy:
    sysno = re.search("<controlfield tag='001'>(\d+)</controlfield>", zaznam, re.DOTALL)
    if sysno == None:
        sysno = ""
    else:
        sysno = sysno.group(1)
    
    ISBN = re.search("<datafield tag='020'.+?<subfield code='a'>((?:[0-9]{3}-)?[0-9]{1,5}-[0-9]{1,7}-[0-9]{1,6}-[0-9])</subfield>", zaznam)
    if ISBN == None:
        ISBN = ""
    else:
        ISBN = ISBN.group(1)
    
    ISSN = re.search("datafield tag='022'.+?<subfield code='a'>([\S]{4}\-[\S]{4})</subfield>", zaznam)
    if ISSN == None:
        ISSN = ""
    else:
        ISSN = ISSN.group(1)
    
    nazev = re.search("<datafield tag='245'.+?<subfield code='a'>(.+?)</subfield>", zaznam)
    if nazev == None:
        nazev = ""
    else:
        nazev = nazev.group(1)
    
    vysledek = {"sysno": sysno, "ISBN": ISBN, "ISSN" : ISSN, "nazev": nazev}
    zaznamy_aleph = zaznamy_aleph.append(vysledek, ignore_index=True)


In [ ]:
zaznamy_aleph.to_csv("Záznamy_aleph.csv", index=False)
zaznamy_aleph.to_excel("Záznam_aleph.xlsx")

In [ ]:
#kontrola výstupu, vytvoření dataframu pro další práci
df=pd.read_csv("Záznamy_aleph.csv", dtype = str)
df.head()

# Soubor s ISBN

In [ ]:
#vytvoření datafrmamu pro ISBN záznamy - mažeme "-" a "978"
ISBN = df.loc[df['ISBN'].notnull()]
ISBN['norma'] = ISBN['ISBN'].str.replace('-','').str.replace(r'^978', '')

# #vybíráme pouze 10 nebo 13 znaků
ISBN_clear =ISBN[ISBN.norma.apply(lambda x: len(str(x))== 10 or (len(str(x))== 13 and x.startswith('979')))]
# del ISBN_clear["type"]
 
# #mažeme poslední znak    
ISBN_clear['norma'] = ISBN_clear.norma.str[:-1]
ISBN_clear.drop(ISBN_clear[ISBN_clear["norma"]=="000000000"].index, inplace=True)

# #deduplikace
ISBN_clear.drop_duplicates(subset=["norma","nazev"], keep="first", inplace=True)
ISBN_clear.rename(columns={"id" : "isbn"}, inplace=True)
#ISBN_clear.head()

ISBN_aleph= ISBN_clear[["sysno", "ISBN", "nazev", "norma"]]
# #ISBN_clear.to_csv('ISBN_2022_FF.csv', encoding='utf-8', index=False)
# #ISBN_clear.info()
# ISBN_clear.info()

ISBN_aleph.head()
ISBN_aleph.to_csv("ISBN_aleph.csv", index=False)

# Soubor s ISSN

In [ ]:
ISSN = df.loc[df['ISSN'].notnull()]
ISSN.head()

In [ ]:
ISSN['norma'] = ISSN['ISSN'].str.replace('-','')
ISSN.drop_duplicates(subset=["norma","nazev"], keep="first", inplace=True)

ISSN_aleph= ISSN[["sysno", "ISSN", "nazev", "norma"]]
# discovery_ISSN["zdroj"]= "discovery"
ISSN_aleph.head()

ISSN_aleph.to_csv("ISSN_aleph.csv", index=False)